Model Training
1.1 Import Data and Required Packages
Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [198]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#modeling

from sklearn.metrics import mean_squared_error , r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings



Import the CSV Data as Pandas DataFrame

In [199]:
df=pd.read_csv('data/stud.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/stud.csv'

Show Top 5 Records

In [ ]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preparing X and Y variables

In [ ]:
X=df.drop(columns=['math_score'],axis=1)

In [ ]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [ ]:
for col in df.columns : 
    print(f"{col} contains :" , end="  ")
    print("Unique Values:" ,df[col].unique())
tab=['gender','race_ethnicity','parental_level_of_education','lunch','test_preparation_course']
for i in tab :
    print(f"{i} contains unique variables : "  , end="  ")
    print(df[i].unique())

gender contains :  Unique Values: ['female' 'male']
race_ethnicity contains :  Unique Values: ['group B' 'group C' 'group A' 'group D' 'group E']
parental_level_of_education contains :  Unique Values: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
lunch contains :  Unique Values: ['standard' 'free/reduced']
test_preparation_course contains :  Unique Values: ['none' 'completed']
math_score contains :  Unique Values: [ 72  69  90  47  76  71  88  40  64  38  58  65  78  50  18  46  54  66
  44  74  73  67  70  62  63  56  97  81  75  57  55  53  59  82  77  33
  52   0  79  39  45  60  61  41  49  30  80  42  27  43  68  85  98  87
  51  99  84  91  83  89  22 100  96  94  48  35  34  86  92  37  28  24
  26  95  36  29  32  93  19  23   8]
reading_score contains :  Unique Values: [ 72  90  95  57  78  83  43  64  60  54  52  81  53  75  89  32  42  58
  69  73  71  74  70  65  87  56  61  84  55  44  41  85  59  17  39  80
 

In [ ]:
y=df['math_score']

In [ ]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

Create Column Transformer with 3 types of transformers

In [ ]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer 

numeric_transformer = StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [ ]:
X=preprocessor.fit_transform(X)
X.shape
X_toSave = pd.DataFrame(X)
X_toSave.to_csv(r'C:\Users\baryo\mlproject\notebook\X_toSave', index=False)#to visualize the data we put it in a file
X_toSave.head(1000).to_excel(r'C:\Users\baryo\mlproject\notebook\X_toSave_Ex.xlsx', index=False, engine='openpyxl')


separate dataset into train and test


In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape , X_test.shape , y_train.shape , y_test.shape

((800, 19), (200, 19), (800,), (200,))

Create an Evaluate Function to give all metrics after model Training

In [ ]:
def evaluate_model(true , predicted):
    mae = mean_absolute_error(true , predicted)
    mse = mean_squared_error(true , predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true , predicted)
    return mae , rmse ,r2_square

In [204]:
models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso() , 
    "Ridge" : Ridge() , 
    "K-Neighbors Regressor" : KNeighborsRegressor() ,
    "Decision Tree" : DecisionTreeRegressor() ,
    "Random Forest Regressor" : RandomForestRegressor() , 
    "XGBRegressor" : XGBRegressor() , 
    "CatBoost Regressor" : CatBoostRegressor(verbose=False),
    "AdaBoost Regressor" : AdaBoostRegressor()

}
model_list = []
r2_list = []
mae_list = []
rmse_list = []
 
for i in range(len(list(models))) : 
    model = list(models.values())[i]
    model.fit(X_train , y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse , model_train_r2_square = evaluate_model(y_train, y_train_pred)
    model_test_mae , model_test_rmse , model_test_r2_square = evaluate_model(y_test , y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for training set')
    print("- root Mean Squared Error : {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error  : {:.4f}".format(model_train_mae))
    print("-R2 Score : {:.4f}".format(model_train_r2_square))

    print('-------------------------------------------------------')

    print("Model performance for Test set ")
    print("- root Mean Squared Error : {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error  : {:.4f}".format(model_test_mae))
    print("-R2 Score : {:.4f}".format(model_test_r2_square))
    r2_list.append(model_test_r2_square)
    mae_list.append(model_test_mae)
    rmse_list.append(model_test_rmse)
    print('='*35)
    print('\n')


LinearRegression
Model performance for training set
- root Mean Squared Error : 5.3231
- Mean Absolute Error  : 4.2667
-R2 Score : 0.8743
-------------------------------------------------------
Model performance for Test set 
- root Mean Squared Error : 5.3940
- Mean Absolute Error  : 4.2148
-R2 Score : 0.8804


Lasso
Model performance for training set
- root Mean Squared Error : 6.5938
- Mean Absolute Error  : 5.2063
-R2 Score : 0.8071
-------------------------------------------------------
Model performance for Test set 
- root Mean Squared Error : 6.5197
- Mean Absolute Error  : 5.1579
-R2 Score : 0.8253


Ridge
Model performance for training set
- root Mean Squared Error : 5.3233
- Mean Absolute Error  : 4.2650
-R2 Score : 0.8743
-------------------------------------------------------
Model performance for Test set 
- root Mean Squared Error : 5.3904
- Mean Absolute Error  : 4.2111
-R2 Score : 0.8806


K-Neighbors Regressor
Model performance for training set
- root Mean Squared Err

In [218]:
R2tab=pd.DataFrame(list(zip(model_list,r2_list)), columns=['Model Name' , 'R2_score']).sort_values(by=["R2_score"],ascending=False)
ametab=pd.DataFrame(list(zip(model_list,mae_list)), columns=['Model Name' , 'mae_score']).sort_values(by=["mae_score"],ascending=True)
rmsetab=pd.DataFrame(list(zip(model_list,rmse_list)), columns=['Model Name' , 'rmse_score']).sort_values(by=["rmse_score"],ascending=True)

alltab = rmsetab=pd.DataFrame(list(zip(model_list,rmse_list,mae_list,r2_list)), columns=['Model Name' , 'rmse_score' , 'mae_score' , 'R2 score']).sort_values(by=["rmse_score"],ascending=True)

print(alltab)

print("-----------------------------------------------------")

print(R2tab)
print("-----------------------------------------------------")
print(ametab)
print("-----------------------------------------------------")
print(rmsetab)
    

                Model Name  rmse_score  mae_score  R2 score
2                    Ridge    5.390387   4.211101  0.880593
0         LinearRegression    5.393994   4.214763  0.880433
8       AdaBoost Regressor    5.885913   4.568606  0.857630
7       CatBoost Regressor    6.008632   4.612532  0.851632
5  Random Forest Regressor    6.042119   4.688032  0.849974
6             XGBRegressor    6.473307   5.057731  0.827797
1                    Lasso    6.519695   5.157882  0.825320
3    K-Neighbors Regressor    7.249400   5.609000  0.784030
4            Decision Tree    7.889233   6.200000  0.744225
-----------------------------------------------------
                Model Name  R2_score
2                    Ridge  0.880593
0         LinearRegression  0.880433
8       AdaBoost Regressor  0.857630
7       CatBoost Regressor  0.851632
5  Random Forest Regressor  0.849974
6             XGBRegressor  0.827797
1                    Lasso  0.825320
3    K-Neighbors Regressor  0.784030
4            

For both AME and RMSE lower value indicates better performance , while for R2 Score the nearest to 1 the better performance , note that R2 Score goes from 0 to 1 so as we can see , the best model to be applied is Ridge , then linear regression and the order is the same for all indicators

Linear Regression

In [ ]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train,y_train)
y_pred = lin_model.predict()


LinearRegression()
